In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
X_balance_scale = pd.read_csv('data/X_balance_scale', delimiter=',')
y_balance_scale = pd.read_csv('data/y_balance_Scale', delimiter=',')

X_car_eval = pd.read_csv('data/X_car_eval', delimiter=',')
y_car_eval = pd.read_csv('data/y_car_eval', delimiter=',')

X_contraceptive = pd.read_csv('data/X_contraceptive', delimiter=',')
y_contraceptive = pd.read_csv('data/y_contraceptive', delimiter=',')

X_dermatology = pd.read_csv('data/X_dermatology', delimiter=',')
y_dermatology = pd.read_csv('data/y_dermatology', delimiter=',')

X_glass = pd.read_csv('data/X_glass', delimiter=',')
y_glass = pd.read_csv('data/y_glass', delimiter=',')

X_hayes = pd.read_csv('data/X_hayes', delimiter=',')
y_hayes = pd.read_csv('data/y_hayes', delimiter=',')

X_heart = pd.read_csv('data/X_heart', delimiter=',')
y_heart = pd.read_csv('data/y_heart', delimiter=',')

X_new_thyroid = pd.read_csv('data/X_new_thyroid', delimiter=',')
y_new_thyroid = pd.read_csv('data/y_new_thyroid', delimiter=',')

X_page = pd.read_csv('data/X_page', delimiter=',')
y_page = pd.read_csv('data/y_page', delimiter=',')

X_pen_based = pd.read_csv('data/X_pen_based', delimiter=',')
y_pen_based = pd.read_csv('data/y_pen_based', delimiter=',')

X_shuttle = pd.read_csv('data/X_shuttle', delimiter=',')
y_shuttle = pd.read_csv('data/y_shuttle', delimiter=',')

X_vertebra = pd.read_csv('data/X_vertebra', delimiter=',')
y_vertebra = pd.read_csv('data/y_vertebra', delimiter=',')

X_wine = pd.read_csv('data/X_wine', delimiter=',')
y_wine = pd.read_csv('data/y_wine', delimiter=',')

X_yeast = pd.read_csv('data/X_yeast', delimiter=',')
y_yeast = pd.read_csv('data/y_yeast', delimiter=',')

X_fars = pd.read_csv('data/X_fars', delimiter=',')
y_fars = pd.read_csv('data/y_fars', delimiter=',')

In [3]:
X_list = [
    X_balance_scale, X_car_eval, X_contraceptive,
    X_dermatology, X_glass, X_hayes, X_heart,
    X_new_thyroid, X_page, X_pen_based, X_shuttle,
    X_vertebra, X_wine, X_yeast, X_fars
]

y_list = [
    y_balance_scale, y_car_eval, y_contraceptive,
    y_dermatology, y_glass, y_hayes, y_heart,
    y_new_thyroid, y_page, y_pen_based, y_shuttle,
    y_vertebra, y_wine, y_yeast, y_fars
]

name_list = [
    'Balance Scale', 'Car Evaluation', 'Contraceptive',
    'Dermatology', 'Glass', 'Hayes', 'Heart',
    'New Thyroid', 'Page', 'Pen Based', 'Shuttle',
    'Vertebra', 'Wine', 'Yeast', 'FARS'
]

In [4]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.ensemble import RUSBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
# from adacost import *


from utils import *
from tqdm import tqdm

In [5]:
clf_dict = {
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42),
    'AdaBoost_SAMME': AdaBoostClassifier(n_estimators=100, algorithm='SAMME', random_state=42),
    #'GradientBoosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.3, max_features=2, max_depth=5, random_state=42),
    #'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    # 'CatBoost': CatBoostClassifier(verbose=0), 
    'RUSBoostClassifier': RUSBoostClassifier(n_estimators=200, random_state=42),
    # 'AdaCost' : AdaCost(algorithm = "SAMME.R",random_state = 100)
}

Define different base learner for each appoarch

In [17]:
base_learners = [{'estimator' : DecisionTreeClassifier(max_depth=1)},
                 {'estimator' : LogisticRegression(random_state=42)},
                 {'estimator' : GaussianNB()}]
name_learners= ['DecisionTreeClassifier','LogisticRegression','GaussianNB']

In [18]:
clf_score_dict = {}

for clf_name, clf in tqdm(clf_dict.items()):
    score = {}
    # base_learners = base_learners_dict[clf_name]
    
    for i, (X, y) in enumerate(zip(X_list, y_list)):

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        for n , base_learner in enumerate(base_learners):
            clf_b = clf.set_params(**base_learner)
            clf_b.fit(X_train, y_train) 
        
            y_preds = clf_b.predict(X_test) 
            y_scores = clf_b.predict_proba(X_test)

            score_p = metric_list(y_test, y_preds, y_scores, len(y.iloc[:,0].unique()))
            score[f'{name_list[i]}_{name_learners[n]}'] = score_p
    
    df = pd.DataFrame.from_dict(score, orient='index', columns=['accuracy', 'precision', 'recall', 'f1','Macro-Averaged AUPRC',
                                                            'F_measure Beta=10','Modified mcc','MCC metirc','Gmean'])
    clf_score_dict[clf_name] = df

100%|██████████| 3/3 [12:12<00:00, 244.00s/it]


In [19]:
clf_score_dict['RUSBoostClassifier']

,accuracy,precision,recall,f1,Macro-Averaged AUPRC,F_measure Beta=10,Modified mcc,MCC metirc,Gmean
Balance Scale_DecisionTreeClassifier,0.912000,0.914672,0.912000,0.912281,0.716417,0.912242,0.793152,0.848966,0.881972
Balance Scale_LogisticRegression,0.840000,0.932780,0.840000,0.869107,0.683262,0.847665,0.729763,0.765907,0.693374
Balance Scale_GaussianNB,0.200000,0.920721,0.200000,0.212341,0.661442,0.215323,0.209108,0.204261,0.012327
Car Evaluation_DecisionTreeClassifier,0.846821,0.848332,0.846821,0.844034,0.703941,0.846958,0.688469,0.676854,0.497303
Car Evaluation_LogisticRegression,0.780347,0.860427,0.780347,0.802204,0.575529,0.787006,0.639075,0.632807,0.596382
Car Evaluation_GaussianNB,0.583815,0.803152,0.583815,0.611026,0.748844,0.598678,0.426562,0.394882,0.109227
Contraceptive_DecisionTreeClassifier,0.444068,0.523261,0.444068,0.454830,0.443577,0.450263,0.199300,0.203306,0.173959
Contraceptive_LogisticRegression,0.515254,0.540372,0.515254,0.519484,0.526925,0.517441,0.277845,0.277089,0.288732
Contraceptive_GaussianNB,0.410169,0.388538,0.410169,0.389975,0.391034,0.408104,0.051984,0.055675,0.046449
Dermatology_DecisionTreeClassifier,0.891892,0.834658,0.891892,0.854470,0.991461,0.886366,0.854084,0.868796,0.000000


In [24]:
file_name_list = []

In [25]:
def save_result(result,file_name_list):
    for clf, score in result.items():
        score.to_csv(f'result/baselearner/{clf}.csv', index=True)
        file_name_list.append(f'{clf}.csv')

In [26]:
save_result(clf_score_dict,file_name_list)

In [27]:
file_name_list

['AdaBoost.csv', 'AdaBoost_SAMME.csv', 'RUSBoostClassifier.csv']